<a href="https://colab.research.google.com/github/aysucengiz/ceng463-hw2/blob/task1/task1/CENG463_hw2_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1
Given a parliamentary speech in one of several languages, identify the ideology of the speaker’s party. In other words, this involves performing binary classification to determine whether the speaker’s party leans left (0) or right (1).


* Select only one country (excluding the UK) and use the parliamentary debates from that country to complete the assigned tasks.

* Fine-tune a **multilingual masked language model:**
  - multilingual BERT
  - XLMRoberta-base
  - language-specific models like Turkish BERT or German BERT.

* In addition, you are required to experiment with a multilingual causal language model:
  - Llama-3.1-8B


https://sidih.si/cdn/121/index.html

## Prepare the Data

Steps:
- Download
- Read with pandas
- Split


### Download and Read w/ Pandas

Download from the given lin in the homework pdf and unzip

In [8]:
!wget -O trainingset-ideology-power.zip https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1

--2024-12-22 09:59:37--  https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813899321 (776M) [application/octet-stream]
Saving to: ‘trainingset-ideology-power.zip’

r.zip                23%[===>                ] 180.58M  26.7MB/s    eta 25s    ^C


In [9]:
!unzip trainingset-ideology-power.zip -d trainingset-ideology-power

Archive:  trainingset-ideology-power.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of trainingset-ideology-power.zip or
        trainingset-ideology-power.zip.zip, and cannot find trainingset-ideology-power.zip.ZIP, period.


Read with pandas and print the head

In [10]:
import pandas as pd
from tabulate import tabulate

data_df = pd.read_csv("./trainingset-ideology-power/orientation/orientation-tr-train.tsv", delimiter="\t")
print(data_df.head())

        id                           speaker sex  \
0  tr00000  ca2031caa4032c51980160359953d507   M   
1  tr00001  4cee0addb3c69f6866869b180f90d45f   M   
2  tr00002  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr00003  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr00004  be82a4ade406ec6774a0a2e38f6957e3   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  24’üncü Yasama Dönemimizin tüm milletvekilleri...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      1  
1  Mr. President, members of lawmakers, as I spea...      1  
2  Mr. President, I'm here to share with you the ...      1  
3  Mr. President, under the principles determined...      1  
4  Mr. President, dear lawmakers, I ask 

### Split

If you intend to participate in the shared task, you can evaluate your model on the test set.
Since the test dataset lacks labels, you must split your training data into **90% for training** and
**10% for testing**. Ensure the split is performed in a `stratified` manner to maintain the proportion
of labels in both subsets.

[Split guide](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.train_test_split.html) from the pdf.

[Stratify guide](https://stackoverflow.com/questions/34842405/parameter-stratify-from-method-train-test-split-scikit-learn) from the pdf.

## Masked LM

Fine-tune the selected masked language model for each task: For one task use ”text_en” and
for the other task use ”text” (original language).

## Causal LM

For each task, perform inference using the selected causal language model twice: Once using
”text en” and once using ”text” (original language).